This is a watered-down version of project 4 from the [fintech bootcamp at home](https://github.com/whitgroves/fintech-bootcamp-at-home) used with some models from my [optiver Kaggle submission](https://github.com/whitgroves/optiver-trading-at-the-close) to test the ensemble's performance. Feel free to treat this as a setup guide if you need an example and/or use case.

In [1]:
!echo $PATH # if /usr/local/cuda/bin is missing, re-run VScode form terminal

/mnt/Data/Repos/clique-ml/.cuda/bin:/usr/local/cuda/bin:/home/whitgroves/.local/bin:/usr/local/cuda/bin:/home/whitgroves/.local/bin:/usr/local/cuda/bin:/home/whitgroves/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin


In [2]:
!ptxas --version # expecting 12.2

ptxas: NVIDIA (R) Ptx optimizing assembler
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:13:58_PDT_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0


In [3]:
import pandas as pd
import random as r

data = pd.read_csv('./stock_bars.csv')
data = data.loc[data.symbol == r.choice(data.symbol.unique())] # pick a single stock from the test data each time
data

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,BB,2022-01-03 05:00:00+00:00,9.32,9.3900,9.1000,9.37,5267494.0,26372.0,9.268915
1,BB,2022-01-04 05:00:00+00:00,9.37,9.4900,9.1200,9.33,6360118.0,29899.0,9.271508
2,BB,2022-01-05 05:00:00+00:00,9.27,9.3471,8.7800,8.78,9620376.0,39131.0,9.026742
3,BB,2022-01-06 05:00:00+00:00,8.79,9.0200,8.4700,8.78,11505077.0,47090.0,8.771278
4,BB,2022-01-07 05:00:00+00:00,8.97,9.2300,8.8000,8.86,6509053.0,29903.0,8.953267
...,...,...,...,...,...,...,...,...,...
511,BB,2024-01-17 05:00:00+00:00,3.33,3.3500,3.2748,3.34,3264012.0,9790.0,3.322546
512,BB,2024-01-18 05:00:00+00:00,3.37,3.4200,3.3450,3.39,3564418.0,9148.0,3.377668
513,BB,2024-01-19 05:00:00+00:00,3.38,3.4550,3.3450,3.43,7640902.0,13106.0,3.423459
514,BB,2024-01-22 05:00:00+00:00,3.60,3.6000,3.6000,3.60,2332.0,15.0,3.569584


In [4]:
def standardize(data:pd.DataFrame, skip_cols:list[str]=[]) -> pd.DataFrame:
    skip_cols = [col for col in skip_cols if col in data.columns]
    skip = data[skip_cols]
    temp = data.drop(skip_cols, axis=1)
    temp = (temp - temp.mean()) / temp.std(ddof=0) # standardize
    temp = temp.ffill().fillna(0) # impute
    return pd.concat([skip, temp], axis=1, join='inner')

norm = standardize(data, ['symbol', 'timestamp'])
norm

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,BB,2022-01-03 05:00:00+00:00,3.230217,3.114700,3.236637,3.282115,-0.326388,0.809768,3.194762
1,BB,2022-01-04 05:00:00+00:00,3.268630,3.189574,3.252447,3.251254,-0.062726,1.157147,3.196758
2,BB,2022-01-05 05:00:00+00:00,3.191805,3.082579,2.983688,2.826914,0.724010,2.066417,3.008389
3,BB,2022-01-06 05:00:00+00:00,2.823042,2.837666,2.738644,2.826914,1.178808,2.850308,2.811788
4,BB,2022-01-07 05:00:00+00:00,2.961328,2.994901,2.999498,2.888637,-0.026787,1.157541,2.951844
...,...,...,...,...,...,...,...,...,...
511,BB,2024-01-17 05:00:00+00:00,-1.371634,-1.407688,-1.367984,-1.370193,-0.809850,-0.823411,-1.381475
512,BB,2024-01-18 05:00:00+00:00,-1.340904,-1.355277,-1.312493,-1.331617,-0.737359,-0.886643,-1.339053
513,BB,2024-01-19 05:00:00+00:00,-1.333221,-1.329071,-1.312493,-1.300756,0.246341,-0.496815,-1.303813
514,BB,2024-01-22 05:00:00+00:00,-1.164205,-1.220503,-1.110925,-1.169596,-1.596929,-1.786162,-1.191358


In [5]:
y = norm['low'].shift(-1) - norm['low'] # since data is standardize, we attempt to learn/predict the difference between t-1 and t
X = norm.drop(['symbol', 'timestamp'], axis=1)

In [6]:
X

,open,high,low,close,volume,trade_count,vwap
0,3.230217,3.114700,3.236637,3.282115,-0.326388,0.809768,3.194762
1,3.268630,3.189574,3.252447,3.251254,-0.062726,1.157147,3.196758
2,3.191805,3.082579,2.983688,2.826914,0.724010,2.066417,3.008389
3,2.823042,2.837666,2.738644,2.826914,1.178808,2.850308,2.811788
4,2.961328,2.994901,2.999498,2.888637,-0.026787,1.157541,2.951844
...,...,...,...,...,...,...,...
511,-1.371634,-1.407688,-1.367984,-1.370193,-0.809850,-0.823411,-1.381475
512,-1.340904,-1.355277,-1.312493,-1.331617,-0.737359,-0.886643,-1.339053
513,-1.333221,-1.329071,-1.312493,-1.300756,0.246341,-0.496815,-1.303813
514,-1.164205,-1.220503,-1.110925,-1.169596,-1.596929,-1.786162,-1.191358


In [7]:
y

0      0.015809
1     -0.268758
2     -0.245044
3      0.260854
4     -0.229235
         ...   
511    0.055491
512    0.000000
513    0.201569
514   -0.047428
515         NaN
Name: low, Length: 516, dtype: float64

In [8]:
# cutoff = int(len(X)*0.8) # 80:20 train/test split
# X_train, X_test = X[:cutoff], X[cutoff:-1]
# y_train, y_test = y[:cutoff], y[cutoff:-1]

# print(len(X), len(X_train), len(X_test))
# print(len(y), len(y_train), len(y_test))

In [9]:
# import clique
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import tensorflow as tf

layers = tf.keras.layers
Sequential = tf.keras.Sequential
regularizer = tf.keras.regularizers.l1(0.001)
tf.keras.utils.set_random_seed(626)

2025-07-19 19:38:45.040336: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-19 19:38:45.048441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752971925.057105   42492 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752971925.059752   42492 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752971925.066613   42492 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [10]:
# N_FEATURES = len(X.columns)
# ACTIVATION_1 = 'tanh' # inputs are standardized so keep negative range
# ACTIVATION_2 = 'relu' # performed better than tanh, sigmoid
# DROPOUT = 0.5         # performed better than 0.3, 0.4

shared_kw = dict(random_state=626, learning_rate=0.2, max_depth=3, subsample=0.8)
xgb_lgb_kw = dict(n_jobs=16, colsample_bytree=0.85, reg_alpha=500)
xgb_cat_kw = dict(early_stopping_rounds=5)
lgb_cat_kw = dict(num_leaves=8, min_child_samples=2000)

models = [ # order matters if limit is set; frontloading stronger models will cause more rejections; the reverse will oversaturate
    xgb.XGBRegressor(**shared_kw, **xgb_lgb_kw, **xgb_cat_kw, eval_metric='mae', tree_method='hist', gamma=0.2), #, nthread=1),
    xgb.XGBRegressor(**shared_kw, **xgb_lgb_kw, **xgb_cat_kw, eval_metric='mae', tree_method='hist', gamma=0.2),
    xgb.XGBRegressor(**shared_kw, **xgb_lgb_kw, **xgb_cat_kw, eval_metric='mae', tree_method='hist', gamma=0.2),
    lgb.LGBMRegressor(**shared_kw, **xgb_lgb_kw, **lgb_cat_kw, early_stopping_round=5, metric='l1', min_split_gain=0.001, verbosity=-1),
    cat.CatBoostRegressor(**shared_kw, **xgb_cat_kw, **lgb_cat_kw, eval_metric='MAE'),
    Sequential([layers.Dense(1, activation='tanh', input_shape=[len(X.columns)])], name='linear'), # N -> 1
    # Sequential([ # N -> N/2 -> 1
    #     layers.Dense(N_FEATURES, kernel_regularizer=regularizer, activation=ACTIVATION_1, input_shape=[N_FEATURES]),
    #     layers.Dropout(DROPOUT),
    #     layers.BatchNormalization(),
    #     layers.Dense(N_FEATURES//2, kernel_regularizer=regularizer, activation=ACTIVATION_2),
    #     layers.Dropout(DROPOUT),
    #     layers.BatchNormalization(),
    #     layers.Dense(1)
    # ], name='net'),
]

/mnt/Data/Repos/clique-ml/.cuda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
import clique
# clique.ModelProfile(models[3])
# clique.Clique(models=models)
# pass
very_exclusive_group = clique.Clique(models=models)
for model_id in very_exclusive_group.keys():
    print(model_id, '\t', very_exclusive_group[model_id])


XGBRegressor 	 <ModelProfile (XGBRegressor)>
XGBRegressor_0 	 <ModelProfile (XGBRegressor)>
XGBRegressor_1 	 <ModelProfile (XGBRegressor)>
LGBMRegressor 	 <ModelProfile (LGBMRegressor)>
CatBoostRegressor 	 <ModelProfile (CatBoostRegressor)>
Sequential 	 <ModelProfile (Sequential)>


In [13]:
very_exclusive_group.targets = y
very_exclusive_group.targets = y.dropna()
very_exclusive_group.targets

0      0.015809
1     -0.268758
2     -0.245044
3      0.260854
4     -0.229235
         ...   
510   -0.051538
511    0.055491
512    0.000000
513    0.201569
514   -0.047428
Name: low, Length: 515, dtype: float64

In [15]:
very_exclusive_group.inputs = X.iloc[:-1]

In [16]:
very_exclusive_group.reset_testing_data(X, y)

In [17]:
very_exclusive_group.targets

0      0.015809
1     -0.268758
2     -0.245044
3      0.260854
4     -0.229235
         ...   
511    0.055491
512    0.000000
513    0.201569
514   -0.047428
515         NaN
Name: low, Length: 516, dtype: float64

In [18]:
def testing_function_test_function() -> None: pass

clique.Clique().scoring = testing_function_test_function

In [ ]:

# # try: ensemble = e.load_ensemble(X_test=X, y_test=y)
# # except FileNotFoundError: 
# ensemble = clique.train_ensemble(models, X, y, folds=5, limit=3)
# ensemble

In [ ]:
# ensemble.best_model

In [ ]:
# ensemble.predict(X)

In [ ]:
len(dict())

0